TVAE Runner (SDV-based)

This script trains a TVAE model on single-cell (tabular) data using the SDV library. It mirrors the CTGAN workflow and supports the same three study scenarios used in the paper.

PBMC3K:

Trains TVAE on real train samples with 100 epochs.

Generates synthetic samples equal to TEST sample size.

Saves: pbmc3k_TVAE_100epc.pkl

HCA-BM10K (5-fold CV)

Integrated Pancreatic dataset (5-fold CV)

For EACH FOLD:

Fit TVAE on TRAIN ONLY.

Generate synthetic samples to reach the Q3 (75th percentile) count of the corresponding cell-type distribution
(per-class if --label-col is provided; labels are assigned accordingly).

Augment TRAIN with synthetic data. VALIDATION/TEST are NEVER touched.

Saves per-fold files under: {output}/folds/fold_{i}/

dictionary includes each fold objects: train_gen, y_train_gen

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install sdv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.1/198.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

In [ ]:
import sdv
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.single_table import CopulaGANSynthesizer

from sdv.metadata import Metadata
from sdv.sampling import Condition as cd

PBMC3K

In [ ]:
import pickle
with open(f"data/pbmc3k_train.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(f"data/pbmc3k_test.pkl", "rb") as f:
    X_test = pickle.load(f)
with open(f"data/pbmc3k_y_train.pkl", "rb") as f:
    y_train = pickle.load(f)
with open(f"data/pbmc3k_y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

5CV Pancrea

In [ ]:
import pickle

all_folds = []

for fold in range(1, 6):
    with open(f"data/5CV/fold_skf_3000_{fold}.pkl", "rb") as f:
        fold_data = pickle.load(f)
        all_folds.append(fold_data)


In [ ]:
for i,f in enumerate(all_folds, start=1):
  print(f"Fold {i}:")
  print("X_train shape:", f['X_train'].shape)
  print("y_train shape:", f['y_train'].shape)

Fold 1:
X_train shape: (11337, 3000)
y_train shape: (11337,)
Fold 2:
X_train shape: (11337, 3000)
y_train shape: (11337,)
Fold 3:
X_train shape: (11338, 3000)
y_train shape: (11338,)
Fold 4:
X_train shape: (11338, 3000)
y_train shape: (11338,)
Fold 5:
X_train shape: (11338, 3000)
y_train shape: (11338,)


In [ ]:
unique_values, counts = np.unique(all_folds[0]['y_train'], return_counts=True)
display(dict(zip(unique_values, counts)),np.max(counts))

{'PSC': np.int64(41),
 'acinar': np.int64(1089),
 'activated_stellate': np.int64(231),
 'alpha': np.int64(3876),
 'beta': np.int64(2975),
 'delta': np.int64(740),
 'ductal': np.int64(1353),
 'endothelial': np.int64(229),
 'epsilon': np.int64(16),
 'gamma': np.int64(350),
 'macrophage': np.int64(48),
 'mast': np.int64(23),
 'mesenchymal': np.int64(69),
 'pp': np.int64(150),
 'quiescent_stellate': np.int64(135),
 'schwann': np.int64(12)}

np.int64(3876)

HCA

In [ ]:
import pickle

all_folds = []

for fold in range(1, 6):
    with open(f"HCA/5CV/fold_skf_{fold}.pkl", "rb") as f:
        fold_data = pickle.load(f)
        all_folds.append(fold_data)



TVAE

In [ ]:
from sdv.single_table import TVAESynthesizer

def tVAE_method(X_min,num_synthetic_samples, metadata):
  tvae = TVAESynthesizer(metadata,epochs=100,verbose=True)
  tvae.fit(X_min)
  # Create synthetic data
  synthetic_data = tvae.sample(num_synthetic_samples)
  return synthetic_data

PBMC3K without class info

In [ ]:
metadata = Metadata()
metadata = metadata.detect_from_dataframe(X_train)
numeric_cols = list(X_train.columns)
for col in numeric_cols:
  metadata.update_column(
  column_name=col,
  sdtype='numerical'
  )
synthetic_samples = tVAE_method(X_train, int(X_test.shape[0]),metadata )

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:129: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss: -8162.612: 100%|██████████| 100/100 [16:06<00:00,  9.66s/it]


In [ ]:
synthetic_samples.shape

(264, 2000)

In [ ]:
with open(g"results" + os.sep + f'pbmc3k_TVAE_100epc.pkl', 'wb') as f:
          pickle.dump(synthetic_samples, f)

5CV

In [ ]:

def TVAE_CV():
  gen_dict = []
  #for k, fold in enumerate(all_folds, start=1):
  for k, fold in enumerate(all_folds, start=1):
      X_train = fold['X_train']
      X_val = fold['X_val']
      y_train = fold['y_train']
      y_val = fold['y_val']

      metadata = Metadata()
      metadata = metadata.detect_from_dataframe(X_train)
      numeric_cols = list(X_train.columns)
      for col in numeric_cols:
          metadata.update_column(
              column_name=col,
              sdtype='numerical'
          )
      # Get unique values and their counts
      unique_values, counts = np.unique(y_train, return_counts=True)
      classlabel_counts = dict(zip(unique_values, counts))
      Q1, Q2, Q3 = np.quantile(counts, [0.25, 0.5, 0.75], axis=0, method='nearest')
      max_count = Q3 #np.max(counts)

      i=1
      for label, count in classlabel_counts.items():
          #print(x.shape, y.shape)
          #print("label, count, max_count",label,count,max_count)
          X_minority = X_train[y_train == label]
          if count < max_count:
              #print("\n")
              print("label, count, max_count",label,count,max_count)
              num_synthetic_samples = max_count - count
              synthetic_samples = tVAE_method(X_minority, num_synthetic_samples,metadata )

              X_minority = np.array(X_minority)
              synthetic_samples = np.array(synthetic_samples)
              if i==1:
                X_train_gen = X_minority
                X_train_gen = np.vstack([X_train_gen, synthetic_samples])

                y_train_gen = np.full(max_count, label)

                y_train_indexes = np.full(count, 1)
                y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
              else:
                tmp = np.vstack([X_minority, synthetic_samples])
                X_train_gen = np.vstack([X_train_gen, tmp])
                y_train_gen = np.concatenate([y_train_gen, np.full(max_count, label)])

                y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
                y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
          else:
              X_train_gen = np.vstack([X_train_gen, X_minority])
              y_train_gen = np.concatenate([y_train_gen, np.full(count, label)])
              y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
          i=i+1

      syn = {
          'X_train_gen': X_train_gen,
          'y_train_gen': y_train_gen
      }
      gen_dict.append(syn)

      with open(gdrivePath + os.sep + "Revision" + os.sep + "HCA/5CV" + os.sep + f'TVAE_skf_fold'+str(k)+'.pkl', 'wb') as f:
          pickle.dump(syn, f)
      print(f"TVAE results for fold {k} saved.")

  return gen_dict



In [ ]:
gen_dict = TVAE_CV()

label, count, max_count 1 476 670


/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:133: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss: -7264.120: 100%|██████████| 100/100 [04:50<00:00,  2.90s/it]


label, count, max_count 2 399 670


Loss: -14066.626: 100%|██████████| 100/100 [04:41<00:00,  2.82s/it]


label, count, max_count 6 275 670


...

label, count, max_count 8 470 670


Loss: -11370.841: 100%|██████████| 100/100 [04:44<00:00,  2.84s/it]


label, count, max_count 10 202 670


Loss: -13477.799: 100%|██████████| 100/100 [04:44<00:00,  2.84s/it]


label, count, max_count 11 377 670
